In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# Import AnimalShelter class from animalShelter
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

#username and password 
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Grazioso Salvare’s logo
image_filename = '/home/paulpathana_snhu/Desktop/GSL.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())



app.layout = html.Div([

    html.Center(html.B(html.H1('Grazioso Search & Rescue Dog Identifier '))),
    html.Center(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '200px', 'width': '200px'}), href='http://www.snhu.edu', target='_blank')),
    html.H1('Paul Pathana', style={'fontSize': '18px'}),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='radio-items',
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',  # Default 
            labelStyle={'display': 'inline-block'}
        )
    ]),
#interactive filtering radio options
    
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[],
                         page_action="native",
                         page_current=0,
                         page_size=10,
    
                         

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', 'data'),
    [Input('radio-items', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'Water':
        query = {
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',  
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}  
        }
    elif filter_type == 'Mountain':
        query = {
            'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',  
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}  
        }
    elif filter_type == 'Disaster':
        query = {
            'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}  
        }
    else:
        query = {}  #

    df_filtered = pd.DataFrame.from_records(shelter.read(query))
    return df_filtered.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # Add code to create a chart based on the viewData
    if viewData:
        # Convert the current data table (viewData) into a DataFrame
        dff = pd.DataFrame.from_dict(viewData)
        # Create a figure using Plotly Express
        fig = px.pie(dff, names='breed')
        # Return the graph component with the figure
        return [dcc.Graph(figure=fig)]
    else:
        # If there's no data, you can return an empty list or a default figure
        return []
   


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # This callback will highlight selected columns in the data table
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, selected_rows):  
    # Check if viewData or selected_rows are empty
    if not viewData or not selected_rows:
        # Return the default map if no data or no row is selected
        return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")])]

    # Convert viewData to DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Check if the selected row index is within the bounds of the DataFrame
    if selected_rows[0] < len(dff):
        
        # Get the data for the selected row
        row = dff.iloc[selected_rows[0]]
       
        # latitude and longitude values
        latitude = row['location_lat']
        longitude = row['location_long']
       
    # Create the map with the marker
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[latitude, longitude], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[latitude, longitude], children=[
                    dl.Tooltip(row['breed']),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(row['name'])
                    ])
                ])
            ])
        ]
    else:
        # Return the default map 
        return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")])]



app.run_server(debug=True)


Connection Successful
Dash app running on http://127.0.0.1:13220/
